In [11]:
import argparse
import json
import os
import time

import yaml

import igibson
from igibson.envs.behavior_mp_env import BehaviorMPEnv
from igibson.utils.ig_logging import IGLogReader

In [12]:
from enum import IntEnum


"""
States list

Burnt, --> Burn
CleaningTool, 
HeatSourceOrSink, 
Cooked, --> Cook
Dusty,  --> True->False: Clean
Frozen, --> Freeze/Unfreeze
Open,   
Sliced, --> False->True: Slice
Slicer, 
Soaked, --> Soak/Dry
Stained, --> True->False: Stain
ToggledOn, --> Toggle
WaterSource, 
InFOVOfRobot, 
InHandOfRobot, 
InReachOfRobot, 
InSameRoomAsRobot, 
InRoom, 
Inside, 
NextTo, 
OnFloor, 
OnTop, 
Touching, 
Under 

"""
class ActionPrimitives(IntEnum):
    NAVIGATE_TO = 0
    LEFT_GRASP = 1
    RIGHT_GRASP = 2
    LEFT_PLACE_ONTOP = 3
    RIGHT_PLACE_ONTOP = 4
    LEFT_PLACE_INSIDE = 5
    RIGHT_PLACE_INSIDE = 6
    OPEN = 7
    CLOSE = 8
    BURN=9
    COOK=10
    CLEAN=11
    FREEZE=12
    UNFREEZE=13
    SLICE=14
    SOAK=15
    DRY=16
    STAIN=17
    TOGGLE=18

In [16]:
segmentation_path = r"C:\Users\bryan\Code\demo1\out\test_motion_planner\bottling_fruit_0_Wainscott_0_int_0_2021-09-15_18-30-15_goal.json"

actions = get_actions_from_segmentation(segmentation_path)

actions

Conversion of demo segmentation to motion primitives:
ActionPrimitives.OPEN(fridge_97)
ActionPrimitives.OPEN(bottom_cabinet_no_top_80)
ActionPrimitives.CLOSE(bottom_cabinet_no_top_80)
ActionPrimitives.OPEN(bottom_cabinet_no_top_80)
ActionPrimitives.CLOSE(bottom_cabinet_no_top_80)
ActionPrimitives.SLICE(strawberry_0_multiplexer)
ActionPrimitives.SLICE(peach_0_multiplexer)
ActionPrimitives.OPEN(jar_0)
Placed object peach_0_multiplexer in segment 8 not currently grasped. Maybe some sort of segmentation error?
ActionPrimitives.CLOSE(jar_0)
ActionPrimitives.OPEN(jar_1)
Placed object strawberry_0_multiplexer in segment 11 not currently grasped. Maybe some sort of segmentation error?
ActionPrimitives.CLOSE(jar_1)
Conversion complete.



[(<ActionPrimitives.OPEN: 7>, 'fridge_97'),
 (<ActionPrimitives.OPEN: 7>, 'bottom_cabinet_no_top_80'),
 (<ActionPrimitives.CLOSE: 8>, 'bottom_cabinet_no_top_80'),
 (<ActionPrimitives.OPEN: 7>, 'bottom_cabinet_no_top_80'),
 (<ActionPrimitives.CLOSE: 8>, 'bottom_cabinet_no_top_80'),
 (<ActionPrimitives.SLICE: 14>, 'strawberry_0_multiplexer'),
 (<ActionPrimitives.SLICE: 14>, 'peach_0_multiplexer'),
 (<ActionPrimitives.OPEN: 7>, 'jar_0'),
 (<ActionPrimitives.CLOSE: 8>, 'jar_0'),
 (<ActionPrimitives.OPEN: 7>, 'jar_1'),
 (<ActionPrimitives.CLOSE: 8>, 'jar_1')]

In [14]:
def get_actions_from_segmentation(segmentation_path):
    # Load the segmentation of a demo for this task.
    with open(segmentation_path, "r") as f:
        demo_data = json.load(f)

    # Get the actions from the segmentation
    
    print("Conversion of demo segmentation to motion primitives:")

    hand_by_object = {}
    actions = []
    segmentation = demo_data["sub_segments"]

    # Convert the segmentation to a sequence of state changes.
    state_changes = []
    for segment in segmentation:
        state_records = segment["state_records"]
        if len(state_records) == 0:
            print("Found segment with no useful state changes: %r" % segment)
            continue
        elif len(state_records) > 1:
            print("Found segment with multiple state changes, using the first: %r" % segment)

        state_change = state_records[0]
        state_changes.append(state_change)

    # Now go through the state changes and convert them to actions
    for i, state_change in enumerate(state_changes):
        # Handle the combinations that we support.
        state_name = state_change["name"]
        state_value = state_change["value"]

        # TODO(replayMP): Here we compute grasps based on the InHand state. Ditch this and simply do a single-hand
        # grasp on the object we will manipulate next. That way it will be fetch-compatible.
        if state_name == "Open" and state_value is True:
            primitive = ActionPrimitives.OPEN
            target_object = state_change["objects"][0]
        elif state_name == "Open" and state_value is False:
            primitive = ActionPrimitives.CLOSE
            target_object = state_change["objects"][0]
        elif state_name == "InReachOfRobot" and state_value is True:
            # The primitives support automatic navigation to relevant objects.
            continue
        elif state_name == "InHandOfRobot" and state_value is True:
            target_object = state_change["objects"][0]

            # Check that we do something with this object later on, otherwise don't grasp it.
            is_used = False
            for future_state_change in state_changes[i + 1 :]:
                # We should only grasp the moved object in these cases.
                if future_state_change["objects"][0] != target_object:
                    continue

                if future_state_change["name"] == "InHandOfRobot" and future_state_change["value"] is True:
                    # This object is re-grasped later. No need to look any further than that.
                    break

                # We only care about Inside and OnTop use cases later.
                if future_state_change["name"] not in ("Inside", "OnTop") or future_state_change["value"] is False:
                    continue

                # This is a supported use case so we approve the grasp.
                is_used = True
                break

            # If the object is not used in the future, don't grasp it.
            if not is_used:
                continue

            hand = get_empty_hand(hand_by_object)
            hand_by_object[target_object] = hand
            primitive = ActionPrimitives.LEFT_GRASP if hand == "left_hand" else ActionPrimitives.RIGHT_GRASP
        elif state_name == "Inside" and state_value is True:
            placed_object = state_change["objects"][0]
            target_object = state_change["objects"][1]
            if placed_object not in hand_by_object:
                print(
                    "Placed object %s in segment %d not currently grasped. Maybe some sort of segmentation error?"
                    % (placed_object, i)
                )
                continue
            hand = hand_by_object[placed_object]
            del hand_by_object[placed_object]
            primitive = (
                ActionPrimitives.LEFT_PLACE_INSIDE if hand == "left_hand" else ActionPrimitives.RIGHT_PLACE_INSIDE
            )
        elif state_name == "OnTop" and state_value is True:
            placed_object = state_change["objects"][0]
            target_object = state_change["objects"][1]
            if placed_object not in hand_by_object:
                print(
                    "Placed object %s in segment %d not currently grasped. Maybe some sort of segmentation error?"
                    % (placed_object, i)
                )
                continue
            hand = hand_by_object[placed_object]
            del hand_by_object[placed_object]
            primitive = ActionPrimitives.LEFT_PLACE_ONTOP if hand == "left_hand" else ActionPrimitives.RIGHT_PLACE_ONTOP
        
        
        elif state_name == "Sliced" and state_value is True:
            primitive = ActionPrimitives.SLICE
            target_object = state_change["objects"][0]
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:     
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:     
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        # elif state_name == "OnTop" and state_value is True:
        
        
        
        else:
            raise ValueError("Found a state change we can't process: %r" % state_change)

        # Append the action.
        action = (primitive, target_object)
        actions.append(action)
        print("%s(%s)" % action)

    print("Conversion complete.\n")
    return actions